In [1]:
from autowig import autowig

In [2]:
from path import path

rootdir = path('../../StatisKit').abspath()

In [3]:
autowig.scons(rootdir + '/misc', '-j3')

In [4]:
autowig.scons(rootdir + '/core', '-j3')

In [5]:
from autowig.tools import subclasses
from autowig.asg import FundamentalTypeProxy

for cls in subclasses(FundamentalTypeProxy):
    if hasattr(cls, '_node'):
        print cls._node

::_Complex double
::long double
::wchar_t
::_Complex long double
::int
::unsigned char
::char
::char32_t
::signed char
::unsigned short int
::unsigned long long int
::long int
::double
::void
::unsigned long int
::bool
::short int
::float
::unsigned int
::char16_t
::nullptr_t
::_Complex float
::long long int


In [6]:
includedir = rootdir + "/core/build-scons/include"

headers = [str(f) for f in includedir.walkfiles('*.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I' + str(includedir)]

asg = autowig.AbstractSemanticGraph()

In [7]:
headers

['/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/base.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/error.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/estimator.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/sample_space.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/slope_heuristic.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/metric.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/mixture.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/distribution.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/predictor.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/moment.h',
 '/home/pfernique/Desktop/StatisKit/core/build-scons/include/statiskit/core/event.h',
 '/home/pfernique/D

In [9]:
includedir = rootdir + "/core/build-scons/include"

headers = [str(f) for f in includedir.walkfiles('*.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I' + str(includedir)]

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags,
                  overload='class', bootstrap=3, permissive=True)

[<pyclanglite.__pyclanglite.clang.NamespaceDecl object at 0x7f6e79de1c20>, <pyclanglite.__pyclanglite.clang.NamespaceDecl object at 0x7f6e79de17c0>, <pyclanglite.__pyclanglite.clang.NamespaceDecl object at 0x7f6e9160d050>, <pyclanglite.__pyclanglite.clang.NamespaceDecl object at 0x7f6e9160d0c0>, <pyclanglite.__pyclanglite.clang.NamespaceDecl object at 0x7f6e9160d130>, <pyclanglite.__pyclanglite.clang.CXXRecordDecl object at 0x7f6e9160d1a0>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d280>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d2f0>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d360>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d3d0>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d440>, <pyclanglite.__pyclanglite.clang.TypedefDecl object at 0x7f6e9160d4b0>, <pyclanglite.__pyclanglite.clang.CXXRecordDecl object at 0x7f6e9160d520>, <pyclanglite.__pyclanglite.clang.TypedefDecl obje

In [10]:
len(asg)

4161

In [11]:
asg.nodes('::statiskit.*')

[::statiskit::get_inf,
 ::statiskit::UnivariateDataFrame::randomize,
 ::statiskit::WeightedUnivariateDataFrame::remove_event,
 ::statiskit::MultivariateEvent::~MultivariateEvent,
 ::statiskit::ContinuousEvent::operator=,
 ::statiskit::get_NR,
 ::statiskit::get_NN,
 ::statiskit::outcome_type::MIXED,
 ::statiskit::interval_error::interval_error,
 ::statiskit::MultivariateSampleSpace::operator=,
 ::statiskit::UnivariateSampleSpace::is_compatible,
 ::statiskit::parameter_error::parameter_error,
 ::statiskit::WeightedUnivariateDataFrame::set_weight,
 ::statiskit::NominalSampleSpace::get_encoding,
 ::statiskit::outcome_type::CONTINUOUS,
 ::statiskit::test_error::operator=,
 ::statiskit::VariableName::VariableName,
 ::statiskit::MultivariateEvent::get_outcome,
 ::statiskit::MultivariateSampleSpace::~MultivariateSampleSpace,
 ::statiskit::OrdinalSampleSpace::operator=,
 ::statiskit::lower_bound_error::lower_bound_error,
 ::statiskit::NominalSampleSpace::~NominalSampleSpace,
 ::statiskit::Ordin

In [10]:
asg.clean()

In [8]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [8]:
asg.clean()

primaries = [header for header in asg.files(header=True) if header.is_primary]
for hdr in asg.files(header=True):
    if not hdr.is_primary and hdr.include in primaries:
        hdr.is_primary = True 
        
for inc in asg.directories():
    if inc.as_include:
        if inc.globalname + 'c++/' in asg:
            for vsn in asg[inc.globalname + 'c++/'].directories:
                vsn.as_include = True
                for hdr in vsn.files:
                    if hdr.suffix == '':
                        hdr.is_primary = True
        elif inc.globalname + 'boost/' in asg:
            for hdr in sg[inc.globalname + 'boost/'].files:
                hdr.is_primary = True
                
for tpl in asg.classes('.*(unique|weak|shared)_ptr$', templated=True,
                       specialized=False):
    tpl.is_smart_pointer = True

AttributeError: 'EnumerationProxy' object has no attribute 'asg'

In [7]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

In [8]:
if 'struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >'].boost_python_export = False
if 'class ::arma::SpMat<double>' in asg:
    for mtd in asg['class ::arma::SpMat<double>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<double>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<double>' in asg:
    for mtd in asg['class ::arma::SpSubview<double>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False
if 'class ::arma::SpMat<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpMat<unsigned long long>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<unsigned long long>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpSubview<unsigned long long>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False

In [9]:
for tdf in  asg['::'].typedefs():
    tdf.boost_python_export = False

In [10]:
from vplants.autowig.boost_python_back_end import BoostPythonExportMappingFileProxy

autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/core/src/wrapper',
                 pattern = '.*statiskit.*',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/misc/src/wrapper',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/wrapper/__core.cpp',
                 package = 'statiskit.core')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/wrapper/__misc.cpp',
                 package = 'statiskit.misc')
autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/statiskit/core/_core.py',
                 module = rootdir + '/core/src/wrapper/__core.cpp')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/statiskit/misc/_misc.py',
                 module = rootdir + '/misc/src/wrapper/__misc.cpp')

In [11]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/(core|misc)/src/(wrapper|statiskit)/.*')

In [12]:
import pickle
with open('/home/pfernique/Desktop/test.pkl', 'w') as f:
    pickle.dump(asg, f)

In [13]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [14]:
asg.__dict__.keys()

['_specialization_edges',
 '_template_edges',
 '_include_edges',
 '_base_edges',
 '_parameter_edges',
 '_nodes',
 '_syntax_edges',
 '_language',
 '_type_edges']

In [15]:
for key, values in asg._specialization_edges.iteritems():
    for value in values:
        if not isinstance(value, basestring):
            print value

In [16]:
for key, values in asg._template_edges.iteritems():
    for value in values:
        if not isinstance(value['target'], basestring):
            print value

In [17]:
for key, value in asg._include_edges.iteritems():
    if not isinstance(value, basestring):
        print value

In [18]:
for key, values in asg._base_edges.iteritems():
    for value in values:
        if not isinstance(value['base'], basestring):
            print value

In [19]:
for key, values in asg._syntax_edges.iteritems():
    for value in values:
        if not isinstance(value, basestring):
            print value

In [20]:
from vplants.autowig.asg import NodeProxy

for key, values in asg._nodes.iteritems():
    for value in values:
        if isinstance(values[value], NodeProxy):
            print key, value

In [21]:
%debug

ERROR: No traceback has been produced, nothing to debug.


In [22]:
autowig.scons(rootdir + '/misc', '-j3')

In [23]:
autowig.scons(rootdir + '/core', '-j3')

In [24]:
asg['::std::string']

KeyError: path(u"'::std::string' parameter")

In [ ]:
asg.nodes('.*string')

In [ ]:
asg.typedefs('.*string')